<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2021-01-07 14:08:04--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip.1’

concrete_data_week4 100%[===================>] 249.37M  1.78MB/s    in 2m 8s   

2021-01-07 14:10:19 (1.95 MB/s) - ‘concrete_data_week4.zip.1’ saved [261483817/261483817]



In [2]:
!unzip concrete_data_week4.zip

Archive:  concrete_data_week4.zip
replace concrete_data_week4/valid/positive/16679_1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [6]:
# 1 Importing libraries 
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense

In [7]:
# 2 Define batch size
batch_size = 100

In [8]:
# 3 Create ImageDataGenerator
image_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
training_image_gen = image_gen.flow_from_directory("./concrete_data_week4/train",
                                                  target_size=(224, 224),
                                                  batch_size=batch_size,
                                                  class_mode='categorical')

validation_image_gen = image_gen.flow_from_directory("./concrete_data_week4/valid",
                                                     target_size=(224, 224),
                                                     batch_size=batch_size,
                                                     class_mode='categorical')

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [9]:
# 4 Create a Keras Sequential model
model = Sequential()
# Add VGG16 model to sequential model
model.add(VGG16(include_top=False, pooling='avg', weights='imagenet'))  # VGG16 model loaded without last layer
# Add a final dense layer to the model
model.add(Dense(2, activation="softmax"))

In [10]:
model.layers[0].trainable = False

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 512)               14714688  
_________________________________________________________________
dense (Dense)                (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [12]:
# 5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])

In [ ]:
# 6. Fit the model on the augmented data using the ImageDataGenerators.
history = model.fit_generator(training_image_gen, 
          steps_per_epoch=len(training_image_gen),
          epochs=2,
          validation_data=validation_image_gen,
          validation_steps=len(validation_image_gen),
          verbose=1
         )

/home/local/SIRIONLABS/anshuman.tiwari1/virtual_environments/ml_venv/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/2
 65/301 [=====>........................] - ETA: 1:56:39 - loss: 0.4154 - accuracy: 0.8131

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [ ]:
# 1. Load your saved model that was built using the ResNet50 model. 
import keras
resnet_model = model  # Model trained above
vgg16_model = keras.models.load_model("classifier_vgg16_model.h5")

In [ ]:
# 2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory 
# of the test images, target size, and the **shuffle** parameter and set it to False.
image_data_gen = ImageDataGenerator()
test_image_gen = image_data_gen.flow_from_directory("./concrete_data_week4/test/", 
                                                    target_size=(224, 224),
                                                    batch_size=batch_size,
                                                    shuffle=False)

In [ ]:
# 3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator 
# as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
# 4. Print the performance of the classifier using the VGG16 pre-trained model.
vgg_16_performance = vgg16_model.evaluate_generator(test_image_gen, verbose=1)
vgg_16_performance

In [ ]:
# 5. Print the performance of the classifier using the ResNet pre-trained model.
resnet_performance = resnet_model.evaluate_generator(test_image_gen, verbose=1)
resnet_performance

<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [ ]:
# 1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data
# ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator**
# method [here](https://keras.io/models/sequential/).

predicted_classes_resnet = resnet_model.predict_generator(test_image_gen, verbose=1)

In [ ]:
predicted_classes_vgg16 = vgg16_model.predict_generator(test_image_gen, verbose=1)

In [ ]:
training_image_gen.class_indices
label_map = {v:k for k ,v in training_image_gen.class_indices.items()}
label_map

In [ ]:
# 2.1. RESNET MODEL - Report the class predictions of the first five images in the test set. You should print something list this:
for entry in predicted_classes_resnet[0:5]:
    max_index = list(entry).index(max(entry))
    print(label_map[max_index])

In [ ]:
# 2.2. VGG16Model - Report the class predictions of the first five images in the test set. You should print something list this:
for entry in predicted_classes_vgg16[0:5]:
    max_index = list(entry).index(max(entry))
    print(label_map[max_index])

### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).